In [83]:
import os
import pandas as pd
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModel


In [85]:
#!pip install einops

In [61]:
def get_sequences(filename):
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def create_sequences(folder_path):
    filenames = []
    for filename in os.listdir(folder_path):
        if '.txt' in filename:
            f = "../../data/" + filename
            filenames.append(f)
    lines = []
    for file in filenames:
        line = get_sequences(file)
        lines = lines + line
    return lines

In [81]:
def create_csv(folder_path, data):
    l = len(data)
    labels = [random.randint(0, 1) for _ in range(l)]
    num_train = int(l * 0.8)
    num_test = ((l - num_train) // 2) + num_train
    
    train_dict = {'sequence': data[:num_train], 'label': labels[:num_train]}
    test_dict ={'sequence': data[num_train:num_test], 'label': labels[num_train:num_test]}
    dev_dict = {'sequence': data[num_test:], 'label': labels[num_test:]}
    
    train_df = pd.DataFrame(train_dict)
    test_df = pd.DataFrame(test_dict)
    dev_df = pd.DataFrame(dev_dict)

    train_path = folder_path + '/train.csv'
    test_path = folder_path + '/test.csv'
    dev_path = folder_path + '/dev.csv'
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    dev_df.to_csv(dev_path, index=False)
    return 'csv files created'

In [82]:
#this creates csv files, only run once if you don't have files
#k = create_sequences(folder_path = "../../data")
#create_csv("../../data/finetuning_data", k[:200])

'csv files created'

In [86]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


bert_padding.py:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- flash_attn_triton.py
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2025-02-12 12:41:22.294795: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: The model class you are passing has a `config_class` attribute that is not consistent with the config class you passed (model has <class 'transformers.models.bert.configuration_bert.BertConfig'> and you passed <class 'transformers_modules.zhihan1996.DNABERT-2-117M.d064dece8a8b41d9fb8729fbe3435278786931f1.configuration_bert.BertConfig'>. Fix one of those so they match!